# Volcado Tabular de Vencimientos Explained con Otros Campos

In [200]:
from datetime import date
import pandas as pd
import requests

## Rutas

In [201]:
rutas = {
    "explained": "https://ms-tesoreria-analytics-staging.interconecta2.cl/v1/settlements/swaps/explained",
    "headers": "https://ms-tesoreria-analytics-staging.interconecta2.cl/operations/all",
}

## SDK

Estas son funciones que utilizan los endpoints adecuados para obtener la data necesaria para el volcado.

In [202]:
def get_explained(fecha: date, ruta: str):
    result = requests.get(f"{ruta}/{fecha}")
    if result.status_code == 200:
        return result.json()
    else:
        return result.status_code

In [203]:
def get_headers(ruta: str):
    result = requests.get(ruta)
    if result.status_code == 200:
        return result.json()
    else:
        return result.status_code

## Procedimiento

Se define la fecha de proceso.

In [204]:
fecha_proceso = date(2025, 6, 12)
f_proc = fecha_proceso.isoformat().replace("-", "")

Se obtienen los explained a fecha de proceso.

In [205]:
explained = get_explained(fecha_proceso, rutas["explained"])

Se obtienen los encabezados de la cartera vigente.

In [206]:
headers = get_headers(rutas["headers"])

Vemos un registro de los explained.

In [207]:
explained[0]

{'dealNumber': '1393',
 'legNumber': 1,
 'tipoPata': 'FIXED_RATE_MCCY',
 'recPay': 'A',
 'fechaInicial': '2025-05-12',
 'fechaFinal': '2025-06-12',
 'fechaPago': '2025-06-12',
 'nocional': 1139.8128,
 'amortizacion': 84.1556,
 'interes': 5.398280066666599,
 'amortEsFlujo': True,
 'flujo': 89.55388006666661,
 'monedaNocional': 'CLF',
 'valorTasa': 0.055,
 'tipoTasa': 'LinAct360',
 'fechaFixingFx': '2025-06-12',
 'monedaPago': 'CLP',
 'indiceFx': 'UF',
 'valorIndiceFx': 39220.03,
 'amortizacionMonedaPago': 3300585.1566680004,
 'interesMonedaPago': 211720.706163066,
 'flujoMonedaPago': 3512305.8628310664,
 'fraccionAgno': '(fechaFinal - fechaInicial) / 360',
 'formulaInteres': 'nocional * valorTasa * fraccionAgno',
 'formulaInteresMonedaPago': 'formulaInteres * valorIndiceFx',
 'formulaAmortizacionMonedaPago': 'amortizacion * valorIndiceFx'}

Se observa que no tiene los campos de portfolio (gestión o contable) ni la estrategia contable. Se los agregamos con la data de los headers.

In [208]:
df_explained = pd.DataFrame.from_dict(explained)

Vemos la estructura de un header.

In [209]:
headers["5000"]

{'trade_date': {'fecha': '2022-11-29'},
 'deal_number': '5000',
 'counterparty': {'name': 'BANCO SANTANDER-SANTIAGO',
  'rut': {'rut': 97036000, 'dv': 'K'},
  'other': None},
 'portfolio': 'BANCA',
 'hedge_accounting': 'YES',
 'product': 'SWAP_MONE',
 'currency_pair': 'CLFCLP',
 'price': None,
 'settlement_mechanism': 'C',
 'other': None}

In [210]:
def generate_new_columns(row):
    # Notar que headers se toma por closure
    operacion = headers.get(row.dealNumber)
    if operacion is None:
        operacion = {"dummy": 1}
    if operacion.get("other") is None:
        regulatory_portfolio = None
    else:
        regulatory_portfolio = operacion.get("other").get("regulatory_portfolio")
    return pd.Series({
        'portfolio': operacion.get("portfolio"),
        'hedgeAccounting': operacion.get("hedge_accounting"),
        'product': operacion.get("product"),
        'settlementMechanism': operacion.get("settlement_mechanism"),
        'regulatoryPortfolio': regulatory_portfolio,
    })

In [211]:
df_explained_ok = df_explained.join(df_explained.apply(
    generate_new_columns,
    axis=1,
))
del df_explained_ok["fraccionAgno"]
del df_explained_ok["formulaValorTasaEquivalente"]
del df_explained_ok["formulaInteres"]
del df_explained_ok["formulaInteresMonedaPago"]
del df_explained_ok["formulaAmortizacionMonedaPago"]

In [212]:
df_explained_ok

,dealNumber,legNumber,tipoPata,recPay,fechaInicial,fechaFinal,fechaPago,nocional,amortizacion,interes,...,valorIndiceFinal,spread,gearing,numDecimalesTasaEquivalente,codigoIndiceTasa,portfolio,hedgeAccounting,product,settlementMechanism,regulatoryPortfolio
0,1393,1,FIXED_RATE_MCCY,A,2025-05-12,2025-06-12,2025-06-12,1.139813e+03,8.415560e+01,5.398280e+00,...,NaN,NaN,NaN,NaN,NaN,SWAP,NO,SWAP_MONE,C,NEGOCIACION
1,1393,2,FIXED_RATE,P,2025-05-12,2025-06-12,2025-06-12,-9.677419e+06,-1.935484e+06,-5.666667e+04,...,NaN,NaN,NaN,NaN,NaN,SWAP,NO,SWAP_MONE,C,NEGOCIACION
2,2349,1,FIXED_RATE_MCCY,A,2025-05-12,2025-06-12,2025-06-12,8.146250e+03,1.045900e+02,2.791901e+01,...,NaN,NaN,NaN,NaN,NaN,SWAP,NO,SWAP_MONE,C,NEGOCIACION
3,2349,2,OVERNIGHT_INDEX,P,2025-05-12,2025-06-12,2025-06-12,-1.993333e+08,-2.888889e+06,1.989213e+08,...,0.0,0.0240,1.0,4.0,NaN,SWAP,NO,SWAP_MONE,C,NEGOCIACION
4,2350,1,FIXED_RATE_MCCY,A,2025-05-12,2025-06-12,2025-06-12,7.931300e+03,1.018200e+02,2.718233e+01,...,NaN,NaN,NaN,NaN,NaN,SWAP,NO,SWAP_MONE,C,NEGOCIACION
5,2350,2,OVERNIGHT_INDEX,P,2025-05-12,2025-06-12,2025-06-12,-1.940625e+08,-2.812500e+06,1.936614e+08,...,0.0,0.0240,1.0,4.0,NaN,SWAP,NO,SWAP_MONE,C,NEGOCIACION
6,6152,1,FIXED_RATE,A,2024-12-12,2025-06-12,2025-06-12,1.500000e+10,0.000000e+00,4.019167e+08,...,NaN,NaN,NaN,NaN,NaN,BANCA,NO,SWAP_ICP,C,NEGOCIACION
7,6152,2,OVERNIGHT_INDEX,P,2024-12-12,2025-06-12,2025-06-12,-1.500000e+10,0.000000e+00,1.499983e+10,...,0.0,0.0000,1.0,4.0,NaN,BANCA,NO,SWAP_ICP,C,NEGOCIACION
8,6153,1,FIXED_RATE_MCCY,A,2024-12-12,2025-06-12,2025-06-12,1.000000e+10,0.000000e+00,2.684500e+08,...,NaN,NaN,NaN,NaN,NaN,BANCA,NO,SWAP_ICP,C,NEGOCIACION
9,6153,2,OVERNIGHT_INDEX_MCCY,P,2024-12-12,2025-06-12,2025-06-12,-1.000000e+10,0.000000e+00,1.000000e+10,...,0.0,0.0000,1.0,6.0,NaN,BANCA,NO,SWAP_ICP,C,NEGOCIACION


In [213]:
#df_explained_ok.to_csv(f"./{f_proc}_explained_plus_plus.csv")

Ahora vamos a convertir la dataframe en una vista por operación.

In [214]:
import numpy as np

Crear una función helper para agregar los datos correctamente

In [215]:
def aggregate_deal_data(group):
    """
    Aggregate deal data from leg-level to deal-level
    """
    # Get deal-level attributes (should be the same for all legs)
    deal_attrs = {
        'DealNumber': group['dealNumber'].iloc[0],
        'product': group['product'].iloc[0],
        'fechaFinal': group['fechaFinal'].iloc[0],
        'settlementMechanism': group['settlementMechanism'].iloc[0],
        'hedgeAccounting': group['hedgeAccounting'].iloc[0],
        'regulatoryPortfolio': group['regulatoryPortfolio'].iloc[0]
    }
    
    # Separate active (A) and passive (P) legs
    active_legs = group[group['recPay'] == 'A']
    passive_legs = group[group['recPay'] == 'P']
    
    # Process active legs
    if len(active_legs) > 0:
        deal_attrs['pataActivaAmortizacion'] = active_legs['amortizacion'].sum()
        deal_attrs['pataActivaMonedaNocional'] = active_legs['monedaNocional'].iloc[0]  # Assuming same currency for all active legs
        # Get fechaPago from lowest legNumber for active legs
        deal_attrs['pataActivaFechaPagoAmortizacion'] = active_legs.loc[active_legs['legNumber'].idxmin(), 'fechaPago']
        # Sum flujoMonedaPago, handling NaN values
        active_flujo_values = active_legs['flujoMonedaPago'].fillna(active_legs['flujo'])
        deal_attrs['pataActivaFlujo'] = active_flujo_values.sum() if not active_flujo_values.isna().all() else np.nan
    else:
        deal_attrs['pataActivaAmortizacion'] = np.nan
        deal_attrs['pataActivaMonedaNocional'] = np.nan
        deal_attrs['pataActivaFechaPagoAmortizacion'] = np.nan
        deal_attrs['pataActivaFlujo'] = np.nan
    
    # Process passive legs
    if len(passive_legs) > 0:
        deal_attrs['pataPasivaAmortizacion'] = passive_legs['amortizacion'].sum()
        deal_attrs['pataPasivaMonedaNocional'] = passive_legs['monedaNocional'].iloc[0]  # Assuming same currency for all passive legs
        # Get fechaPago from lowest legNumber for passive legs
        deal_attrs['pataPasivaFechaPagoAmortizacion'] = passive_legs.loc[passive_legs['legNumber'].idxmin(), 'fechaPago']
        # Sum flujoMonedaPago, handling NaN values
        passive_flujo_values = passive_legs['flujoMonedaPago'].fillna(passive_legs['flujo'])
        deal_attrs['pataPasivaFlujo'] = passive_flujo_values.sum() if not passive_flujo_values.isna().all() else np.nan
    else:
        deal_attrs['pataPasivaAmortizacion'] = np.nan
        deal_attrs['pataPasivaMonedaNocional'] = np.nan
        deal_attrs['pataPasivaFechaPagoAmortizacion'] = np.nan
        deal_attrs['pataPasivaFlujo'] = np.nan
    
    return pd.Series(deal_attrs)

Agrupamos por deal number

In [216]:
df_deal_level = df_explained_ok.groupby('dealNumber').apply(aggregate_deal_data).reset_index(drop=True)

C:\Users\bencl\AppData\Local\Temp\ipykernel_25276\4184180563.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_deal_level = df_explained_ok.groupby('dealNumber').apply(aggregate_deal_data).reset_index(drop=True)


Aplicar conversiones de nombres de producto

In [217]:
# Apply product name conversions
product_mapping = {
    'SWAP_ICP': 'Swap Promedio Cámara'.encode('utf-8').decode('utf-8'),
    'SWAP_MONE': 'Swap Moneda',
    'SWAP_TASA': 'Swap Tasas'
}
df_deal_level['product'] = df_deal_level['product'].map(product_mapping).fillna(df_deal_level['product'])

Y calcular monto neto a compensar en caso de modalidad compensación

In [218]:
df_deal_level['netoACompensar'] = np.nan
mask_compensation = df_deal_level['settlementMechanism'] == 'C'

df_deal_level.loc[mask_compensation, 'netoACompensar'] = (
    df_deal_level.loc[mask_compensation, 'pataActivaFlujo'].fillna(0) + 
    df_deal_level.loc[mask_compensation, 'pataPasivaFlujo'].fillna(0)
)

Y reordenamos las columnas

In [219]:
# Reorder columns to include the new netoACompensar column
column_order = [
    'DealNumber', 'product', 'fechaFinal', 'settlementMechanism', 'hedgeAccounting', 
    'regulatoryPortfolio', 'pataActivaAmortizacion', 'pataActivaMonedaNocional', 
    'pataActivaFechaPagoAmortizacion', 'pataActivaFlujo', 'pataPasivaAmortizacion',
    'pataPasivaMonedaNocional', 'pataPasivaFechaPagoAmortizacion', 'pataPasivaFlujo',
    'netoACompensar'
]

df_deal_level = df_deal_level[column_order]

Y vemos los resultados:

In [220]:
print("Deal-level dataframe shape:", df_deal_level.shape)
print("\nFirst few rows:")
print(df_deal_level.head())

print("\nDataframe info:")
print(df_deal_level.info())

print("\nSample of deal-level data:")
print(df_deal_level.head(10))

Deal-level dataframe shape: (11, 15)

First few rows:
  DealNumber               product  fechaFinal settlementMechanism  \
0       1393           Swap Moneda  2025-06-12                   C   
1       2349           Swap Moneda  2025-06-12                   C   
2       2350           Swap Moneda  2025-06-12                   C   
3       6152  Swap Promedio Cámara  2025-06-12                   C   
4       6153  Swap Promedio Cámara  2025-06-12                   C   

  hedgeAccounting regulatoryPortfolio  pataActivaAmortizacion  \
0              NO         NEGOCIACION                 84.1556   
1              NO         NEGOCIACION                104.5900   
2              NO         NEGOCIACION                101.8200   
3              NO         NEGOCIACION                  0.0000   
4              NO         NEGOCIACION                  0.0000   

  pataActivaMonedaNocional pataActivaFechaPagoAmortizacion  pataActivaFlujo  \
0                      CLF                      2025-06

In [221]:
df_deal_level

,DealNumber,product,fechaFinal,settlementMechanism,hedgeAccounting,regulatoryPortfolio,pataActivaAmortizacion,pataActivaMonedaNocional,pataActivaFechaPagoAmortizacion,pataActivaFlujo,pataPasivaAmortizacion,pataPasivaMonedaNocional,pataPasivaFechaPagoAmortizacion,pataPasivaFlujo,netoACompensar
0,1393,Swap Moneda,2025-06-12,C,NO,NEGOCIACION,84.1556,CLF,2025-06-12,3.512306e+06,-1.935484e+06,CLP,2025-06-12,-1.992151e+06,1.520155e+06
1,2349,Swap Moneda,2025-06-12,C,NO,NEGOCIACION,104.5900,CLF,2025-06-12,5.197007e+06,-2.888889e+06,CLP,2025-06-12,1.960324e+08,2.012294e+08
2,2350,Swap Moneda,2025-06-12,C,NO,NEGOCIACION,101.8200,CLF,2025-06-12,5.059475e+06,-2.812500e+06,CLP,2025-06-12,1.908489e+08,1.959084e+08
3,6152,Swap Promedio Cámara,2025-06-12,C,NO,NEGOCIACION,0.0000,CLP,2025-06-12,4.019167e+08,0.000000e+00,CLP,2025-06-12,1.499983e+10,1.540175e+10
4,6153,Swap Promedio Cámara,2025-06-12,C,NO,NEGOCIACION,0.0000,CLP,2025-06-12,2.859258e+05,0.000000e+00,CLP,2025-06-12,1.065099e+07,1.093691e+07
5,6154,Swap Promedio Cámara,2025-06-12,C,NO,NEGOCIACION,0.0000,CLP,2025-06-12,2.859258e+05,0.000000e+00,CLP,2025-06-12,1.065099e+07,1.093691e+07
6,6156,Swap Promedio Cámara,2025-06-12,C,NO,NEGOCIACION,0.0000,CLP,2025-06-12,4.011583e+08,0.000000e+00,CLP,2025-06-12,1.499983e+10,1.540099e+10
7,6157,Swap Promedio Cámara,2025-06-12,C,NO,NEGOCIACION,0.0000,CLP,2025-06-12,-9.999889e+09,0.000000e+00,CLP,2025-06-12,-2.669333e+08,-1.026682e+10
8,6158,Swap Moneda,2025-06-12,C,NO,NEGOCIACION,0.0000,CLP,2025-06-12,-1.764682e+06,0.000000e+00,USD,2025-06-12,-4.214689e+04,-1.806829e+06
9,6797,Swap Moneda,2025-06-12,C,NO,NEGOCIACION,0.0000,CLF,2025-06-12,1.127209e+05,0.000000e+00,CLP,2025-06-12,9.986471e+06,1.009919e+07


In [222]:
#df_deal_level.to_csv(f"./{f_proc}_deal_level.csv", index=False, encoding='utf-8')
#df_deal_level.to_excel(f"./{f_proc}_deal_level.xlsx", index=False)
df_deal_level.to_excel(rf"C:\Users\bencl\OneDrive - palace.cl\Documents\Palace\Banco Internacional\Treasury PM\06 Pruebas\{f_proc}_deal_level.xlsx", index=False)
